In [14]:
import pandas as pd

In [15]:
df = pd.read_csv(r'C:\Users\manhm\Desktop\Smart_Phone\OKbro.csv')

In [16]:

df.columns


Index(['Tên sản phẩm', 'Giá bán cũ', 'Giá mới', 'condition',
       'Thời điểm ra mắt', 'Thời gian bảo hành (tháng)', 'Phiên bản CPU',
       'RAM', 'Kích thước màn hình', 'Chip đồ hoạ (GPU)', 'Dung lượng (ROM)',
       'Camera', 'Hỗ trợ mạng', 'Bluetooth', 'Dung lượng pin', 'OS'],
      dtype='object')

In [17]:
import pandas as pd
import pyodbc

# Load the CSV file
file_path = r'C:\Users\manhm\Desktop\Smart_Phone\OKbro.csv'
df = pd.read_csv(file_path)

# Clean and ensure data types
df['Giá bán cũ'] = pd.to_numeric(df['Giá bán cũ'], errors='coerce')  # Convert to numeric, invalid values to NaN
df['Giá mới'] = pd.to_numeric(df['Giá mới'], errors='coerce')
df['RAM'] = pd.to_numeric(df['RAM'], errors='coerce')
df['Dung lượng (ROM)'] = pd.to_numeric(df['Dung lượng (ROM)'], errors='coerce')
df['Dung lượng pin'] = pd.to_numeric(df['Dung lượng pin'], errors='coerce')

# Replace NaN values with 0 or some appropriate default value
df.fillna(0, inplace=True)

# Define your connection details
server = 'adylab.database.windows.net'
database = 'ADY_lab'
username = 'adylab'
password = 'manh123456@'
driver = '{ODBC Driver 17 for SQL Server}'

# Create a connection to the Azure SQL database
connection = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}')
cursor = connection.cursor()

# Define the table creation query
create_table_query = '''
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[product_data]') AND type in (N'U'))
BEGIN
    CREATE TABLE product_data (
        TenSanPham NVARCHAR(MAX),
        GiaBanCu FLOAT,
        GiaMoi FLOAT,
        Condition NVARCHAR(MAX),
        ThoiDiemRaMat NVARCHAR(MAX),
        ThoiGianBaoHanh NVARCHAR(MAX),
        PhienBanCPU NVARCHAR(MAX),
        RAM FLOAT,
        KichThuocManHinh NVARCHAR(MAX),
        ChipDoHoaGPU NVARCHAR(MAX),
        DungLuongROM FLOAT,
        Camera NVARCHAR(MAX),
        HoTroMang NVARCHAR(MAX),
        Bluetooth NVARCHAR(MAX),
        DungLuongPin FLOAT,
        OS NVARCHAR(MAX)
    )
END
'''

# Execute the create table query
cursor.execute(create_table_query)
connection.commit()

# Insert data from the DataFrame into the Azure SQL database
for index, row in df.iterrows():
    insert_query = '''
    INSERT INTO product_data (
        TenSanPham, GiaBanCu, GiaMoi, Condition, ThoiDiemRaMat, ThoiGianBaoHanh, 
        PhienBanCPU, RAM, KichThuocManHinh, ChipDoHoaGPU, DungLuongROM, 
        Camera, HoTroMang, Bluetooth, DungLuongPin, OS
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(insert_query, (
        row['Tên sản phẩm'], 
        row['Giá bán cũ'], 
        row['Giá mới'], 
        row['condition'], 
        row['Thời điểm ra mắt'], 
        row['Thời gian bảo hành (tháng)'], 
        row['Phiên bản CPU'], 
        row['RAM'], 
        row['Kích thước màn hình'], 
        row['Chip đồ hoạ (GPU)'], 
        row['Dung lượng (ROM)'], 
        row['Camera'], 
        row['Hỗ trợ mạng'], 
        row['Bluetooth'], 
        row['Dung lượng pin'], 
        row['OS']
    ))

# Commit the inserts
connection.commit()

# Close the connection
cursor.close()
connection.close()

print("Data inserted successfully into the Azure SQL database.")


Data inserted successfully into the Azure SQL database.
